Load all pdfs with the document and directory load

In [ ]:
import os
from langchain_community.document_loaders import PyMuPDFLoader , DirectoryLoader

def load_directory(directory_path:str):
    dir_loader = DirectoryLoader(
        directory_path,
        glob="**/*.pdf",
        loader_cls= PyMuPDFLoader,
        show_progress=False
    )
    directory_documents = dir_loader.load()
    return directory_documents

directory_documents = load_directory("../data/pdf")


19
page_content='Welcome to Bajaj Allianz Family
 BARDOLI(VSO)-Shop Nos.25 To 28, Mezzanine Floor, , Megh Mayur Plazaratan
Baug, , Surat Dumas Roadsurat, Surat, Gujarat, INDIA, 395007, 9999999999
Policy issuing office and correspondence address for communication by
policyholder for claim, service request, notice, summons, etc.:
   Insured Name
Pragneshkumar Rajubhai Nayka
 Policy Number
 12-1805-0007923327-00
Name: Pragneshkumar Rajubhai Nayka
Address:
Line 1: Parsigali Faliyu , Boriya T Mahuva D Surat
City: Surat, State: Gujarat
Postcode: 395620
Customer ID: PI35470101
Line 2:   ,
Dear Pragneshkumar Rajubhai Nayka,
We thank you for choosing Bajaj Allianz for your Insurance needs. We are one of India's leading general insurance companies with iAAA rating
from ICRA for the last ten consecutive years indicating the company's high claims paying ability and fundamentally strong position in the industry.
Please be assured that you have made the right choice by choosing us and we will stand 

Add splitting logic

In [26]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path

def split_documents(directory_documents, chunk_size = 500 , chunk_overlap =100):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = chunk_size,
        chunk_overlap = chunk_overlap,
        length_function = len,
        separators= ["\n\n", "\n"]
    )

    split_docs = text_splitter.split_documents(directory_documents)
    print(f"Split {len(directory_documents)} documents into {len(split_docs)} chunks")

    # if(split_docs):
    #     print("logging...")
    #     print(split_docs[0].metadata)
    #     print(split_docs[0].page_content[:100])
    return split_docs


spilt_doc_to_commit = split_documents(directory_documents, 500, 100)



Split 19 documents into 103 chunks


chunking done. 

embeddings 
convert chunks to embeddings

In [34]:
# imports
import numpy as np 
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Tuple, Dict, Any
from sklearn.metrics.pairwise import cosine_similarity


In [43]:
# create embeddings 
# class structure
class EmbeddingManager:
    def __init__(self, model_name: str= "all-MiniLM-L6-v2"):
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        try:
            print(f"loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully, dimension of embeddings: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading the model {self.model_name}: {e}");
            raise
    def generate_embeddings(self, texts: List[str])-> np.ndarray:
        if not self.model:
            raise ValueError("Model not loaded yet")
        print("Generating embeddings")
        embeddings = self.model.encode(texts, show_progress_bar= True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings